## Project part 2
#### Author: Rahul Gore

In [1]:
'''Imports'''
import scipy
import scipy.io
import numpy as np
from libsvm.svmutil import *
import os
# import time

In [33]:
train_data = scipy.io.loadmat('trainData.mat')
train_x1 = np.array(train_data['X1'])
train_x2 = np.array(train_data['X2'])
train_x3 = np.array(train_data['X3'])
train_y = train_data['Y'].flatten()

In [39]:
test_data = scipy.io.loadmat('testData.mat')
test_x1 = np.array(test_data['X1'])
test_x2 = np.array(test_data['X2'])
test_x3 = np.array(test_data['X3'])
test_y = test_data['Y'].flatten()

### Step 0 (1)
Classification models

In [43]:
''' WARNING: Do this step only once. Load from *.model file '''
class_h1 = svm_train(train_y, train_x1, '-c 10 -t 0 -q')
class_h2 = svm_train(train_y, train_x2, '-c 10 -t 0 -q')
class_h3 = svm_train(train_y, train_x3, '-c 10 -t 0 -q')
svm_save_model('class_h1.model', class_h1)
svm_save_model('class_h2.model', class_h2)
svm_save_model('class_h3.model', class_h3)

In [4]:
class_h1 = svm_load_model('class_h1.model')
class_h2 = svm_load_model('class_h2.model')
class_h3 = svm_load_model('class_h3.model')

In [44]:
label1, acc1, val1 = svm_predict(test_y, test_x1, class_h1)
label2, acc2, val2 = svm_predict(test_y, test_x2, class_h2)
label3, acc3, val3 = svm_predict(test_y, test_x3, class_h3)

Accuracy = 11.3648% (214/1883) (classification)
Accuracy = 17.5252% (330/1883) (classification)
Accuracy = 8.60329% (162/1883) (classification)


In [61]:
''' testing '''
val1 = np.array(val1)
print(val1.shape)

(1883, 1225)


### Step 0 (2)
Probability models

In [45]:
''' WARNING - Takes too long. Prefer to load from *.model file '''

prob_h1 = svm_train(train_y, train_x1, '-c 10 -t 0 -b 1 -q')
prob_h2 = svm_train(train_y, train_x2, '-c 10 -t 0 -b 1 -q')
prob_h3 = svm_train(train_y, train_x3, '-c 10 -t 0 -b 1 -q')
svm_save_model('prob_h1.model', h1)
svm_save_model('prob_h2.model', h2)
svm_save_model('prob_h3.model', h3)

In [46]:
prob_h1 = svm_load_model('prob_h1.model')
prob_h2 = svm_load_model('prob_h2.model')
prob_h3 = svm_load_model('prob_h3.model')

In [47]:
label1, acc1, val1 = svm_predict(test_y, test_x1, prob_h1, "-b 1")
label2, acc2, val2 = svm_predict(test_y, test_x2, prob_h2, "-b 1")
label3, acc3, val3 = svm_predict(test_y, test_x3, prob_h3, "-b 1")

Accuracy = 28.6776% (540/1883) (classification)
Accuracy = 28.3059% (533/1883) (classification)
Accuracy = 28.0404% (528/1883) (classification)


### Step 1

In [47]:
''' Testing '''
val1 = np.array(val1)
val2 = np.array(val2)
val3 = np.array(val3)
print(val1.shape)

(1883, 50)


In [11]:
vals = [val1, val2, val3]
vals = np.array(vals)
stack_val = np.stack(vals, axis=-1)
fusion = np.mean(stack_val, axis=2)
# print(fusion.shape)

In [12]:
fusion_pred = np.argmax(fusion, axis=1)
fusion_pred += 1
# print(fusion_pred.shape)

In [13]:
match = []
for i in range(1883):
    if fusion_pred[i] == test_y[i]:
        match.append(1)
    else:
        match.append(0)
# print(match[0:50])
print("Accuracy: ", sum(match)/18.83)

Accuracy:  44.82209240573553


In [31]:
''' testing '''
print(fusion_pred[0:10])
print(train_y[0:10])

[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]


### Step 2
Combined features

In [37]:
train_features = np.stack((train_x1, train_x2, train_x3), axis=1)
train_features = np.reshape(train_features, (train_features.shape[0], train_features.shape[1]*train_features.shape[2]))
print(train_features.shape)

(4786, 3000)


In [38]:
''' WARNING: Do this step only once. Load from file '''
comb_h1 = svm_train(train_y, train_features, '-c 10 -t 0 -q')
svm_save_model('comb_h1.model', comb_h1)

In [ ]:
comb_h = svm_load_model('comb_h.model')

In [40]:
test_features = np.stack((test_x1, test_x2, test_x3), axis=1)
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1]*test_features.shape[2]))
print(test_features.shape)

(1883, 3000)


In [42]:
label, acc, val = svm_predict(test_y, test_features, comb_h1)

Accuracy = 39.1928% (738/1883) (classification)
